# Predictive Delay Analytics

In [1]:
%matplotlib inline
# import required modules for prediction tasks
import numpy as np
import pandas as pd
import math
import random

## 0. Data acquisition and cleaning

In [2]:
%%time
# reads all predefined months for a year and merge into one data frame
rawData2014 = pd.DataFrame.from_csv('cache/predictionData/complete2014Data.csv')

Wall time: 1min 22s


In [ ]:
print rawData2014.columns
rawData2014.head(5)

### Cleaning the data

When cleaning the data set, we have to remove the following entries:

- flights that have been cancelled or diverted. We focus on predicting the delay. As a result, we also remove the columns associated with diverted flights.
- colmuns that give the answer. This is the case of many colmuns related to the arrival of the plane
- rows where a value is missing

Note that data points have to be cleaned in this order because most flights have empty entries for the 'diverted' columns.

In [ ]:
#entries to be dropped in the analysis
columns_dropped = ['index', 'TAIL_NUM', 'FL_NUM', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', \
                   'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'CANCELLED', 'CANCELLATION_CODE', 'AIR_TIME', \
                   'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']

In [ ]:
def clean(data, list_col):
    ''' 
    Creates a dataset by excluding undesirable columns

    Parameters:
    -----------

    data: pandas.DataFrame
       Flight dataframe  

    list_col: <list 'string'>
        Comumns to exclude from the data set
    '''
    
    data.drop(data[data.CANCELLED == 0].index, inplace=True)
    data.drop(list_col, axis=1, inplace=True)
    data.dropna(axis = 0, inplace = True)
    return

In [ ]:
%%time
data2014 = clean(rawData2014.copy(), columns_dropped)
print data2014.columns

In [ ]:
%%time
# save the data to avoid computing them again
file_path = "cache/predictionData/predictionData2014.csv"
data2014.to_csv(path_or_buf= file_path)

In [21]:
%%time
# recover data2014 from cache/predictionData folder
file_path = "cache/predictionData/predictionData2014.csv"
data2014 = pd.read_csv(file_path)
data2014.drop('Unnamed: 0', axis= 1, inplace = True)
data2014.columns

Wall time: 11.2 s


In [ ]:
# test that clean did the job
print "size of raw data set: ", len(rawData2014)
print "number of cancelled: ", len(rawData2014[(rawData2014.CANCELLED == 1)])
print "size of data set: ", len(data2014)

Let's add an example that we will follow all along. The example looks a ta flight from New York to Chicago.

In [22]:
dexample = data2014[data2014.DEST == 'ORD'][(data2014.ORIGIN == 'JFK')  | (data2014.ORIGIN == 'LGA') | \
                                              (data2014.ORIGIN == 'EWR')].copy()

dexample.head(3)

,FL_DATE,UNIQUE_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,DISTANCE,AIRCRAFT_YEAR,AIRCRAFT_MFR,LAT,LONG
2033,2014-01-01,AA,LGA,ORD,700,850,-17,733,1991,MCDONNELL DOUGLAS,40.766667,-73.866667
2034,2014-01-04,AA,LGA,ORD,700,850,32,733,2007,FRIEDEMANN JON,40.766667,-73.866667
2035,2014-01-05,AA,LGA,ORD,700,850,11,733,2007,FRIEDEMANN JON,40.766667,-73.866667


### Restricting the dataset

The dataset has more than 4 millions entries, which makes any data manipulation extremely costly - let alone model fitting. We will therefore make some restrictions on the airports and the airlines considered.

In [14]:
data2014.groupby('UNIQUE_CARRIER').size()

UNIQUE_CARRIER
AA    137960
AS    157312
B6    225303
DL    644768
EV    616920
F9     81758
FL     69486
HA     73370
MQ      2995
OO    586067
UA    471093
US    381688
VX     57056
WN    597821
dtype: int64

We are only interested in the carrier that operates from New York to Chicago. Looking at the table, we also notice that Atlantic Southeast Airlines (airline code EV) is only marginally present. So we drop it from the list of carriers we will study in addition to the other carriers that do not operate on the line.

In [15]:
dexample.groupby('UNIQUE_CARRIER').size()

UNIQUE_CARRIER
AA     126
B6     955
EV       2
OO     191
UA    6821
dtype: int64

In [16]:
def restrict_carrier(data, droplist):
    ''' 
    Drop carriers from the data set.

    Parameters:
    -----------

    data: pandas.DataFrame
       dataframe  

    droplist: <list 'string'>
        List of carriers to be droppped
    '''        
    
    for item in droplist:
        data.drop(data[data.UNIQUE_CARRIER == item].index, inplace= True)
    return 

In [23]:
%%time
drop_airline = [ 'AS','DL', 'EV', 'F9', 'FL', 'HA', 'MQ', 'US', 'VX', 'WN']
restrict_carrier(data2014, drop_airline)
print "number of points", len(data2014)
print "airlines", set(data2014.UNIQUE_CARRIER)

number of points 1420423
airlines set(['AA', 'OO', 'B6', 'UA'])
Wall time: 10.5 s


We now focus on the main airports. We look for airports that have on average 50 domestic flight everyday.

In [24]:
def restrict_example(data):
    ''' 
    Restrict dataset to airports for the example
    '''  
    
    data.drop(data[(data.ORIGIN != 'JFK') & (data.ORIGIN != 'LGA') & (data.ORIGIN != 'EWR') & \
                  (data.DEST != 'ORD')].index, inplace=True)
    return

In [26]:
%%time
restrict_example(data2014)
print "number of points", len(data2014)

number of points 205309
Wall time: 1.44 s


In [ ]:
#remove all airports that have an annual traffic under threshold

def restrict_airport(data, threshold):
    ''' 
    Drop carriers from the data set.

    Parameters:
    -----------

    data: pandas.DataFrame
       dataframe  

    droplist: <list 'string'>
        List of carriers to be droppped
    '''     
    
    dict_count = data.groupby("DEST").agg(['count']).LAT.to_dict()['count']
    
    for key in dict_count:
        if dict_count[key] < threshold:
            data.drop(data[data.DEST == key].index, inplace=True)
            data.drop(data[data.ORIGIN == key].index, inplace=True)
    
    print data.groupby("DEST").agg(['count']).LAT.to_dict()['count']
    
    return

** WARNING ** \
RUN THIS CELL ONLY ONCE!

In [27]:
%%time
#restrict_airport(data2014, 60*365)
data_example = data2014.copy()
print "number of airports: ", len(set(data2014))
print "dataset size: ", len(data2014)

number of airports:  12
dataset size:  205309
Wall time: 19 ms


In [28]:
%%time
# save the restricted data to avoid computing them again
file_path = "cache/predictionData/data_example.csv"
data_example.to_csv(path_or_buf= file_path)

Wall time: 1.41 s


In [29]:
data_example.head(3)

,FL_DATE,UNIQUE_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DELAY,DISTANCE,AIRCRAFT_YEAR,AIRCRAFT_MFR,LAT,LONG
0,2014-01-01,AA,JFK,LAX,900,1225,13,2475,1987,BOEING,40.633333,-73.783333
1,2014-01-02,AA,JFK,LAX,900,1225,1,2475,1987,BOEING,40.633333,-73.783333
2,2014-01-04,AA,JFK,LAX,900,1225,59,2475,1986,BOEING,40.633333,-73.783333


In [ ]:
#%%time
# recover file
#file_path = "cache/predictionData/dataRes.csv"
#dataRes = pd.read_csv(file_path)
#dataRes.drop('Unnamed: 0', axis= 1, inplace = True)
#print dataRes.columns

### Encoding categorical variables

In [30]:
from time import strptime
days = {0:"Mon", 1:"Tues", 2:"Wed", 3:"Thurs", 4:"Fri", 5:"Sat", 6:"Sun"}
months = {1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"June", 7:"July", 8:"Aug", 9:"Sep", \
          10:"Oct", 11:"Nov", 12:"Dec"}

In [31]:
def adjust_time(data):
    monlist = np.empty(len(data), dtype = str)
    daylist = np.empty(len(data), dtype = str)
    
    for i in xrange(len(data)):
        date= strptime(data.FL_DATE.iloc[i], "%Y-%M-%d")
        monlist[i] = months[date.tm_min]
        daylist[i] = days[date.tm_wday]

    return monlist, daylist

In [34]:
data_example.columns

Index([u'FL_DATE', u'UNIQUE_CARRIER', u'ORIGIN', u'DEST', u'CRS_DEP_TIME',
       u'CRS_ARR_TIME', u'ARR_DELAY', u'DISTANCE', u'AIRCRAFT_YEAR',
       u'AIRCRAFT_MFR', u'LAT', u'LONG'],
      dtype='object')

In [35]:
%%time
monlist, daylist = adjust_time(data_example)
print "OK"
data_example['MONTH'] = pd.Series(monlist, index=data_example.index)
data_example['DAY'] = pd.Series(daylist, index=data_example.index)
if 'FL_DATE' in data_example.columns:
    data_example.drop('FL_DATE', axis = 1, inplace= True)
print data_example.columns

OK
Index([u'UNIQUE_CARRIER', u'ORIGIN', u'DEST', u'CRS_DEP_TIME', u'CRS_ARR_TIME',
       u'ARR_DELAY', u'DISTANCE', u'AIRCRAFT_YEAR', u'AIRCRAFT_MFR', u'LAT',
       u'LONG', u'MONTH', u'DAY'],
      dtype='object')
Wall time: 10.7 s


In [36]:
%%time
monlist, daylist = adjust_time(dexample)
print "OK"
dexample['MONTH'] = pd.Series(monlist, index=dexample.index)
dexample['DAY'] = pd.Series(daylist, index=dexample.index)
if 'FL_DATE' in dexample.columns:
    dexample.drop('FL_DATE', axis = 1, inplace= True)
print dexample.columns

OK
Index([u'UNIQUE_CARRIER', u'ORIGIN', u'DEST', u'CRS_DEP_TIME', u'CRS_ARR_TIME',
       u'ARR_DELAY', u'DISTANCE', u'AIRCRAFT_YEAR', u'AIRCRAFT_MFR', u'LAT',
       u'LONG', u'MONTH', u'DAY'],
      dtype='object')
Wall time: 445 ms


### Adjusting numerical data

Let's change the script to put time in minutes.

In [37]:
%%time
ti = lambda x: x/100*60+x%100
data_example['CRS_ARR_TIME_COR'] = data_example.CRS_ARR_TIME.map(ti)
data_example['CRS_DEP_TIME_COR'] = data_example.CRS_DEP_TIME.map(ti)
data_example.drop(['CRS_DEP_TIME', 'CRS_ARR_TIME'], axis = 1, inplace = True)

before change:  0    1225
1    1225
Name: CRS_ARR_TIME, dtype: int64

after change:  0    745
1    745
Name: CRS_ARR_TIME_COR, dtype: int64
Wall time: 825 ms


In [38]:
%%time
dexample['CRS_ARR_TIME_COR'] = dexample.CRS_ARR_TIME.map(ti)
dexample['CRS_DEP_TIME_COR'] = dexample.CRS_DEP_TIME.map(ti)
dexample.drop(['CRS_DEP_TIME', 'CRS_ARR_TIME'], axis = 1, inplace = True)

Wall time: 92 ms


We need to center and normalize all continuous data

In [39]:
# # change the age of the aircraft from a string type to an integer type
data_example.drop(data_example[data_example.AIRCRAFT_YEAR =='    '].index, inplace = True)
data_example['AIRCRAFT_YEAR_COR'] = data_example.AIRCRAFT_YEAR.map(lambda x: int(x))
data_example.drop('AIRCRAFT_YEAR', axis = 1, inplace = True)

In [40]:
dexample.drop(dexample[dexample.AIRCRAFT_YEAR =='    '].index, inplace = True)
dexample['AIRCRAFT_YEAR_COR'] = dexample.AIRCRAFT_YEAR.map(lambda x: int(x))
dexample.drop('AIRCRAFT_YEAR', axis = 1, inplace = True)

In [41]:
def normalize(array):
    mean = np.mean(array)
    std = np.std(array)
    return [(x - mean)/std for x in array]

In [42]:
def normalize_data(data, feature_list):
    ''' 
    Normalize data.

    Parameters:
    -----------

    data: pandas.DataFrame
       dataframe  

    feature_list: <list 'string'>
        List of features to be normalized
    '''           

    for feature in feature_list:
        if feature in data.columns:
            data[feature + '_NOR'] = normalize(data[feature].values)
            data.drop(feature, axis =1, inplace=True)
    return

In [92]:
%%time
normalize_feature = ['CRS_DEP_TIME_COR', 'CRS_ARR_TIME_COR', 'DISTANCE', 'LONG', 'LAT', 'AIRCRAFT_YEAR_COR']
normalize_data(data_example, normalize_feature)
normalize_data(dexample, normalize_feature)

Wall time: 35 ms


We are only interested in whetehr a flight will be more than 15 minutes late. So we adjust the ARR_DELAY colum to an indicator.

In [44]:
#data_example['ARR_DELAY_COR'] = data_example.ARR_DELAY.map(lambda x: (x >= 15))
#data_example.drop('ARR_DELAY', axis = 1, inplace = True)

### Encode categorical variables

In [45]:
encoded_list = ['UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'AIRCRAFT_MFR', 'MONTH','DAY']

In [46]:
%%time
finalData_example = pd.get_dummies(data_example, columns=encoded_list)

Wall time: 1.84 s


In [48]:
%%time
# save the restricted data to avoid computing them again
file_path = "cache/predictionData/finalData_example.csv"
finalData_example.to_csv(path_or_buf= file_path)

Wall time: 23.8 s


In [49]:
#%%time
# recover data2014 from cache/predictionData folder
#file_path = "cache/predictionData/finalData.csv"
#finalData = pd.read_csv(file_path)
#finalData.drop('Unnamed: 0', axis= 1, inplace = True)

## 1. Baseline classifiers

We will make prediction on the variable 'ARR_DEL15'. This variable takes the value 1 is the plane is more than 15 minutes late and 0 if not. Let's look at the baseline classifier, that is the classifiers that assign repectively 1 or 0 to 'ARR_DEL15' for every flight.

In [50]:
from __future__ import division

def baseline(data, target):
    ''' 
    Compute the baseline classifiers along a target variable for a data set data

    Parameters:
    -----------

    data: pandas.DataFrame
       dataframe  

    target: string
        Column of data along wich we compute the baseline classifiers
    '''    
    
    
    score_baseline_1 = np.size(data[data[target] == 1][target].values) / np.size(data[target].values)
    score_baseline_0 = np.size(data[data[target] == 0][target].values) / np.size(data[target].values)
    
    print "baseline classifier everyone to 0: ", int(score_baseline_0*100) , "%"
    print "baseline classifier everyone to 1: ", int(score_baseline_1*100) , "%"
   
    return score_baseline_0, score_baseline_1

In [51]:
#score_baseline_0, score_baseline_1 = baseline(finalData_example, 'ARR_DELAY_COR')

### Split data into training/test sets

First, let's split the data set into a training set and a test set. 

In [52]:
from sklearn.cross_validation import train_test_split

In [53]:
def split(data, list_drop, target, test_size):
    ''' 
    Splits the data into a training and a test set
    Separates the training and test sets according to a feature set and a target set
    Balance the features sets by retaining only fraction of its points

    Parameters:
    -----------

    data: pandas.DataFrame
       Flight dataframe  

    list_drop: <list 'string'>
        List of columns to exclude from the features set
        
    target: string
        target column along whch we make the target set
        
    test_size: float
        size of the test set
    
    '''    
    
    #split the dataset into a training set and a test set
    dtrain, dtest = train_test_split(data, test_size = 0.3)
    
    Xtrain = dtrain.drop(list_drop, axis=1).values
    ytrain = dtrain[target].values
    Xtest = dtest.drop(list_drop, axis=1).values
    ytest = dtest[target].values
    
    return Xtrain, ytrain, Xtest, ytest

In [54]:
finalData_example.columns

Index([u'ARR_DELAY', u'CRS_DEP_TIME_COR_NOR', u'CRS_ARR_TIME_COR_NOR',
       u'DISTANCE_NOR', u'LONG_NOR', u'LAT_NOR', u'AIRCRAFT_YEAR_COR_NOR',
       u'UNIQUE_CARRIER_AA', u'UNIQUE_CARRIER_B6', u'UNIQUE_CARRIER_OO', 
       ...
       u'MONTH_J', u'MONTH_M', u'MONTH_N', u'MONTH_O', u'MONTH_S', u'DAY_F',
       u'DAY_M', u'DAY_S', u'DAY_T', u'DAY_W'],
      dtype='object', length=227)

In [55]:
Xtrain, ytrain, Xtest, ytest = split(finalData_example, ['ARR_DELAY'], 'ARR_DELAY', 0.4)

## 2. Random Forest

In [79]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [86]:
def score_random_forest(Xtrain, ytrain, Xtest, ytest, n_trees=10, max_features='auto'):
    ''' 
    Fits a random forest with (Xtrain ,ytrain)
    Computes the score on (Xtest, ytest)

    Parameters:
    -----------

    Xtrain: numpy 2D array
       Feature training set

    ytrain: numpy 1D array
        Target training set
    
    Xtest: numpy 2D array
       Feature test set

    ytest: numpy 1D array
        Target test set
    
    n_trees: int
        number of trees in the forest
    
    max_features: string or int
        number of features used for every tree
        
    Outputs:
    --------
    
    score_train: float
        score on the train set
    
    score_test: float
        score on the test set
    
    clf.feature_importances_
        weights of each feature as used by the classifier
    
    ''' 

    clf= RandomForestRegressor(n_estimators=n_trees, max_features= max_features)
    clf.fit(Xtrain, ytrain)
    
    score_train = mean_squared_error(clf.predict(Xtrain), ytrain)
    score_test = mean_squared_error(clf.predict(Xtest), ytest)
    
    return  score_train, score_test, clf

In [87]:
def best_parameters(Xtrain, ytrain, Xtest, ytest, nb_trees, nb_features):
    ''' 
    Fits sequentially random forest classifiers
    Adds each test score in a pandas.DataFrame with the number of trees, the loss function, the train score,
    and the importance of each features
    Returns a DataFrame with all scores

    Parameters:
    -----------

    Xtrain: numpy 2D array
       Feature training set

    ytrain: numpy 1D array
        Target training set
    
    Xtest: numpy 2D array
       Feature test set

    ytest: numpy 1D array
        Target test set
    
    n_trees: <list int>
        list of numbers of trees in the forest
    
    nb_features: <list int>
        list of number of features in the forest
        
    Outputs:
    --------
    
    score_tab: pandas.DataFrame
        DataFrame of scores with associated parameters
    
    '''
    
    score_tab = pd.DataFrame(columns=['nb_trees', 'nb_features', 'test_score', 'train_score', 'classifier'])
    
    # counter will increment the index in score_tab
    counter = 0 

    for n_estimators in nb_trees:
        for max_features in nb_features:

            score_train, score_test, classifier = \
            score_random_forest(Xtrain, ytrain, Xtest, ytest, n_trees=n_estimators, max_features=max_features) 
            score_tab.loc[counter] = [n_estimators, max_features, score_test, score_train, classifier]
            counter += 1

    return score_tab

In [88]:
def classify_random_forest(data, list_drop, target, test_size=0.4, nb_trees=[10], nb_features = ['auto']):
    Xtrain, ytrain, Xtest, ytest = split(data, list_drop, target, test_size)
    scores =  best_parameters(Xtrain, ytrain, Xtest, ytest, nb_trees, nb_features)
    return scores

In [98]:
nb_trees = [50]
nb_features = ['log2']
test_size = 0.4

In [99]:
%%time
randomForest2014 =  classify_random_forest(finalData_example, ['ARR_DELAY'], 'ARR_DELAY', test_size=test_size, nb_trees=nb_trees, nb_features=nb_features)
print randomForest2014.head(3)

   nb_trees nb_features   test_score  train_score  \
0        50        log2  2344.664596   549.668531   

                                          classifier  
0  (DecisionTreeRegressor(criterion='mse', max_de...  
Wall time: 3min 14s


In [ ]:
# save file to /data/ folder
file_path = "cache/predictionData/randomForest2014.csv"
randomForest2014.to_csv(path_or_buf= file_path)

### Important coefficients

Let's now look at the importance coefficients, that the average usage of each coefficients in the random forest.

In [ ]:
best_clf = randomForest2014[randomForest2014.test_score = np.max(randomForest2014.test_score.values)].classifier.values

In [ ]:
Xexample = dexample.drop(list_drop, axis=1).values
yexample = dexample[target].values

### Predictions and ROC curves

*Describe Process*

# 3. Predicting flight delay time with Linear Regression

To allow users to enjoy more than a classfication experience we want to give them an expected delay time in minutes.

In [2]:
%matplotlib inline
# import required modules for prediction tasks
import numpy as np
import pandas as pd
import math
import random
import requests
import zipfile
import StringIO
import re
import json
import os

# sklearn functions used for the linear regression model
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_absolute_error

First, let us again establish a baseline which has to be beaten by our model.

In [3]:
# first step is to load the actual data and exclude rows that are unnecessary
# a script that produces the csv file can be found in the src folder
# (it might take some while to run, on my macbook up to one hour)
print('loading data...')
df = pd.read_csv('cache/BigFlightTable.csv', nrows=2000)

loading data...


As flight delay changes over the time of day, we introduce a new feature which models in a categorical variable 10 Minute time windows. I.e. each window is fitted to have some effect.